In [1]:
!python --version

Python 3.7.12


In [4]:
%%capture

#enable the process bar
!conda install -y -c conda-forge ipywidgets
!jupyter nbextension enable --py widgetsnbextension

!conda install -y pytorch torchaudio cudatoolkit=10.2 -c pytorch
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

In [1]:
%%capture
#!pip install import-ipynb

In [3]:
#import  import_ipynb


In [41]:
!pip install datasets==1.18.3

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 KB 3.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 KB 20.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
  Attempting uninstall: datasets
    Found existing installation: datasets 1.13.3
    Uninstalling datasets-1.13.3:
      Successfully uninstalled datasets-1.13.3


In [4]:
%%capture
!pip install datasets==1.13.3

In [5]:
%%capture
!pip install transformers==4.11.3

In [6]:
%%capture
!pip install torchaudio

In [7]:
%%capture
!pip install librosa

In [10]:
%%capture 
!pip install boto3

In [11]:
import boto3

In [12]:
my_session = boto3.session.Session()
s3 = my_session.client('s3')

In [13]:
# insert AWS keys here

import datasets

In [26]:
s3 = boto3.resource(
service_name='s3',
region_name='eu-west-1',
aws_access_key_id= data['aws_access_key_id'],
aws_secret_access_key= data['aws_secret_access_key'])


In [15]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "tr", split="train+validation",cache_dir='/home/ec2-user/SageMaker/dataset')
common_voice_test = load_dataset("common_voice", "tr", split="test",cache_dir='/home/ec2-user/SageMaker/dataset/')

Downloading:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/621M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /home/ec2-user/SageMaker/dataset/common_voice/tr/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9. Subsequent calls will reuse this data.


Reusing dataset common_voice (/home/ec2-user/SageMaker/dataset/common_voice/tr/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [16]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [17]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [18]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)


  0%|          | 0/3478 [00:00<?, ?ex/s]

  0%|          | 0/1647 [00:00<?, ?ex/s]

In [19]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[î]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[ô]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[û]', 'u', batch["sentence"])
    return batch

common_voice_train = common_voice_train.map(replace_hatted_characters)
common_voice_test = common_voice_test.map(replace_hatted_characters)


  0%|          | 0/3478 [00:00<?, ?ex/s]

  0%|          | 0/1647 [00:00<?, ?ex/s]

In [27]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [21]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
import json


In [28]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)
s3.Object('commonvoicesdataset', 'turkish/vocab.json').put(Body=json.dumps(vocab_dict,indent = 3))

{'ResponseMetadata': {'RequestId': 'YDGGGAZYVMS3ZD56',
  'HostId': '37VAhF4MLGjiuIFfSy/S9bBnMReIKK6k2r2Vit2pFgFdauSfh/SSmBVY3FexHHKQ0pBcu/VixvI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '37VAhF4MLGjiuIFfSy/S9bBnMReIKK6k2r2Vit2pFgFdauSfh/SSmBVY3FexHHKQ0pBcu/VixvI=',
   'x-amz-request-id': 'YDGGGAZYVMS3ZD56',
   'date': 'Wed, 16 Feb 2022 08:58:43 GMT',
   'etag': '"c7d3eb9ebe4d490797fe69350028cbf8"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c7d3eb9ebe4d490797fe69350028cbf8"'}

In [29]:
# this tokenizer is using the previous json file/ so becareful what u load
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [30]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [31]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [32]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test  = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [33]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [34]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)

In [49]:
aws_access_key_id=data['aws_access_key_id']
aws_secret_access_key=data['aws_secret_access_key']
s3  = boto3.resource('s3',aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key)
s3c = boto3.client('s3',aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key)

In [42]:

from datasets.filesystems import S3FileSystem 

ImportError: cannot import name 'S3FileSystem' from 'datasets.filesystems' (/home/ec2-user/anaconda3/envs/stt/lib/python3.7/site-packages/datasets/filesystems/__init__.py)

In [44]:
!pip install s3fs

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 32.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 KB 26.5 MB/s eta 0:00:00
  Created wheel for aiobotocore: filename=aiobotocore-2.1.1-py3-none-any.whl size=55483 sha256=bba75f387baccedfbd6e1f48c2e9a5a438ea3a2588aa7bbca3cfa2c78c46c080
  Stored in directory: /home/ec2-user/.cache/pip/wheels/52/fd/d3/e85dd95a98c12426ed6864dc24a408c3ca3253d4387b2f1cce
Successfully built aiobotocore
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.0
    Uninstalling botocore-1.24.0:
      Successfully uninstalled botocore-1.24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following d

In [45]:
import s3fs

In [ ]:
s3fs.S

In [46]:
s3fsystems = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)  

In [47]:
common_voice_train.save_to_disk('s3://commonvoicesdataset//turkish/train',fs=s3fsystems)

In [48]:
common_voice_test.save_to_disk('s3://commonvoicesdataset/turkish/test',fs=s3fsystems)

In [ ]:
#should also save the json file

In [ ]:
#dataset1 = load_from_disk('s3://commonvoicesdataset/test',fs=s31)  


From https://github.com/Nadian-Ali/STT
 * branch            HEAD       -> FETCH_HEAD
fatal: refusing to merge unrelated histories


new git


In [ ]:
%now we only want to make thos file